In [15]:
from tensorflow.keras.models import load_model
import pickle
from albumentations import ShiftScaleRotate,Cutout,RandomContrast,RandomBrightness,Compose
from utility.albumentations_helper import create_transform
from functions import generate_feature,loop_distance,dot_distance_neg_np
import numpy as np
import pandas as pd
from functools import reduce,partial

In [13]:
## change ##
color = False
FFA_size = 4
pct = 0.1
model_file = 'feature_model_DenseNet_greyscale_cross_entropy'
distanceFun = dot_distance_neg_np

Load model

In [4]:
feature_model = load_model('Models/'+model_file+'.h5')

In [5]:
#feature_model = load_model('Models/feature_l2_distance_margin_loss_fun_factory(0.1,5).h5', \
#                            custom_objects={'loss': margin_loss_fun_factory(0.1,5)})

In [6]:
with open('submit_train.pkl', 'rb') as f:
    submit_train = pickle.load(f)
with open('submit_test.pkl', 'rb') as f:
    submit_test = pickle.load(f)  
with open('submit_map.pkl', 'rb') as f:
    submit_map = pickle.load(f)        

In [7]:
aug = Compose([RandomContrast(p=0.2),RandomBrightness(p=0.2),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=15,scale_limit=0.02,p=1)])
transform = create_transform(aug)    

In [16]:
feature_train = generate_feature(submit_train,transform,FFA_size,color,feature_model)
feature_val = generate_feature(submit_test,transform,FFA_size,color,feature_model)
predicts = loop_distance(feature_train,feature_val,distanceFun,partial(np.quantile,q=pct,axis=(1,2)))

In [17]:
predicts = np.vectorize(submit_map.get)(predicts)

In [20]:
predicts=reduce(np.core.defchararray.add,['new_whale ',predicts[:,0],\
                                             ' ', predicts[:,1],\
                                             ' ', predicts[:,2],\
                                             ' ', predicts[:,3]])

In [21]:
submit_df = pd.DataFrame(list(zip([i[0].split('/')[-1].split('.')[0]+'.jpg' for i in submit_test],\
                          predicts.tolist())), columns=['Image','Id'])

In [22]:
submit_df.to_csv('submissions/'+model_file+'.csv',index=False)